### LEGALST-190 Lab 2/13

---

In this lab, students will learn how to construct a heatmap, as well as an interactive heat map. This will also be a component of the take-home problem set. This builds ontop of the folium labs from last week.


In [19]:
# dependencies
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import json
import os

In [55]:
!pip install folium --upgrade
import folium.plugins # The Folium Javascript Map Library
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

Requirement already up-to-date: folium in c:\users\gibson\appdata\local\programs\python\python36\lib\site-packages
Requirement already up-to-date: requests in c:\users\gibson\appdata\local\programs\python\python36\lib\site-packages (from folium)
Requirement already up-to-date: six in c:\users\gibson\appdata\local\programs\python\python36\lib\site-packages (from folium)
Requirement already up-to-date: branca in c:\users\gibson\appdata\local\programs\python\python36\lib\site-packages (from folium)
Requirement already up-to-date: jinja2 in c:\users\gibson\appdata\local\programs\python\python36\lib\site-packages (from folium)
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in c:\users\gibson\appdata\local\programs\python\python36\lib\site-packages (from requests->folium)
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in c:\users\gibson\appdata\local\programs\python\python36\lib\site-packages (from requests->folium)
Requirement already up-to-date: idna<2.7,>=2.5 in c:\users\gib

---

## The Data <a id='data'></a>
---

Today we'll be working with data on Berkeley crime calls, courtesy of the Berkeley Police department. Take a look at the metadata [here.](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5)

Note: this data set has already undergone a fair amount of cleaning to format it for our purposes (e.g. extracting the longitude and latitude, removing null values, and dropping irrelevant columns). You can see the original data at the source website.

Then, run the cell below to load the data into a Table.  

In [3]:
calls = Table.read_table('data/berkeley_crime_0218.csv', index_col=0)
calls.show(5)

CASENO,OFFENSE,CVLEGEND,BLKADDR,City,State,Day,Lat,Lon,timestamp
17076632,BURGLARY AUTO,BURGLARY - VEHICLE,1300 SAN PABLO AVE,Berkeley,CA,Monday,37.8803,-122.296,2017-12-18 09:45:00
17092227,THEFT MISD. (UNDER $950),LARCENY,1600 SHATTUCK AVE,Berkeley,CA,Monday,37.8781,-122.269,2017-10-30 10:25:00
18004102,BURGLARY COMMERCIAL,BURGLARY - COMMERCIAL,1400 SHATTUCK AVE,Berkeley,CA,Saturday,37.882,-122.27,2018-01-20 05:23:00
17065730,ALCOHOL OFFENSE,LIQUOR LAW VIOLATION,SOLANO AVENUE & COLUSA AVE,Berkeley,CA,Saturday,37.8914,-122.279,2017-10-28 20:08:00
18000630,BURGLARY AUTO,BURGLARY - VEHICLE,800 POTTER ST,Berkeley,CA,Wednesday,37.8513,-122.293,2018-01-03 19:00:00


When working with any new data set, it's a good idea to get to know it first. Use the following cell and the information on cityofberkeley.org to answer some basic questions:
- What information does this table contain? What are the different columns?
- How large is the data set? 
- What kinds of questions could we answer using this data set?

## Heatmap <a id='data'></a>

Let's see if we can figure out what heatmap does and why it is useful.But first, we're going to quickly review how to use folium.Map. Again, you should consult https://python-visualization.github.io/folium/quickstart.html for a refresher in case you forget how folium works!

Plot a map of the United States again using folium.Map.

<b>Reminder</b>: It is in the order of lat, log and the larger the zoom_start is the closer the map is.

In [5]:
# First, we create a folium Map
example_map1 = folium.Map([39.83, -98.59], zoom_start=6)
example_map1

### Key Note

Heatmaps do not take Tables so you will need to provide a list of lat, lons, i.e. a list of lists. 

Imagine that it looks something like this: `[[lat, lon],[lat, lon],[lat, lon],[lat, lon],[lat, lon]]`. This means if you were given a Table, there are a few steps you'd have to take.

1. Make sure the lat and lon are floats.
2. Filter the Table for the correct rows and columns.

What is something else you believe you'll need to check for to make sure that Heatmap will work?

<b> Solution </b> : Check for NaNs. 

Our data set today has already had the NaNs filtered out, but that might not be true for data you work with in the future...

Run the next cell to generate a set of dummy `[[lat, lon]]` pairs for the HeatMap. Don't worry about the information itself. Instead, note how the array is formatted.

In [6]:
# The first two lines generate an array of small random numbers.
# The third line adds the random numbers to the pair [48, 5] to get 100 latitude, longitude pairs near [48, 5]
data = (np.random.normal(size=(100, 2)) *
        np.array([[1, 1]]) +
        np.array([[39.83, -98.59]])).tolist()
data

[[42.03357062437661, -98.00540483192718],
 [39.6455674903634, -97.4423102600839],
 [41.39500552630867, -98.55989814078356],
 [41.48135986344858, -97.84980422996347],
 [38.98279048165866, -98.94018650657686],
 [39.44630446783679, -98.64755740909962],
 [39.811994862976384, -96.718260472208],
 [39.140077685797245, -98.82371194873267],
 [40.212928012808334, -98.21323812159602],
 [40.7875899578474, -98.15406278717897],
 [39.09718256875883, -98.68996613416559],
 [39.395376562632684, -98.92101842498431],
 [39.1084652786168, -99.85397883385743],
 [41.65783616377477, -96.72190752863676],
 [40.168394280602925, -99.0348344251184],
 [38.65674967438515, -100.04860386917998],
 [39.96500613954399, -97.5194056679859],
 [38.848178099261894, -98.71713646804164],
 [39.90732582135295, -99.75618166365915],
 [39.506691597689695, -97.1332594865037],
 [40.63808179059089, -99.23680947746067],
 [38.95458570772039, -98.63285533776823],
 [41.20177833422214, -99.13133385528985],
 [40.944805208712715, -100.73796333

Then we can plot it on the map! The function is pretty simple: 
1. Create a Heatmap using the function `Heatmap(your_lat_lon_data)`
2. Add that Heatmap to your existing map with `add_to(your_map)`

In [8]:
# Add the HeatMap to the map
HeatMap(data).add_to(example_map1)

example_map1

Play around with your new Heatmap. What is it plotting? What kinds of things would a Heatmap be useful for?

<b> Solution</b>: 

### Try It Out

Now, try making your own Heatmap using the Berkeley PD call data. First, plot a Folium Map of the Bay Area, just like you did last week.

In [9]:
#Plot the map of Berkeley
berk_coords = (37.87, -122.27)
berk_map = folium.Map(location=berk_coords, zoom_start=13)
berk_map

Next, extract your latitude and longitude data from the `calls` Table and save each to the variables `lat` and `lon`. We want the data as a numpy array, so don't use the `select` function; instead, index the Table by the correct column (e.g. `calls["Column_I_Want"]`).

In [10]:
lat = calls['Lat']
lon = calls['Lon']
lat

array([37.880262, 37.878112, 37.881957, ..., 37.87718 , 37.858132,
       37.862763])

We have the right data, but it isn't in the right shape: we want an array of arrays, where the first column is latitudes, the second column is longitudes, and each row is a `[lat, lon]` pair (see the example above). We can do this by:
1. **Stacking** the `lat` array on top of the `lon` array into one larger array with `np.vstack`
2. **Transposing** our stacked array so the latitude and longitude are vertical columns, not horizontal rows.

Hint 1: the stacking function call looks something like `np.vstack((top_array, bottom_array))`
Hint 2: you can transpose an array by calling `.transpose()` on the array

In [42]:
call_locs = np.vstack((lat, lon)).transpose().tolist() # Solution
call_locs[1]

[37.878112, -122.26911399999999]

Now, you have everything you need to make your HeatMap! Do so in the cell below.

In [43]:
#Create a Heatmap with the call data.
heatmap = HeatMap(call_locs, radius = 10) # Solution

# Add it to your Berkeley map.
berk_map.add_child(heatmap) #Solution

What conclusions can you draw from this Heatmap?

<b>Solution</b>:

## HeatMapwithTime <a id='data'></a>

Now what do you think is different with HeatMapwithTime?

<b> Solution</b>: It is a list of list of lists! Latitude/longitude pairs are grouped into lists by their shared dates.

In this example, we'll again use dummy data to show how it works. It follows a similar process to HeatMap. First, create another Folium Map centered at the geographical center of the USA.

In [44]:
# Create a folium Map at the USA's center
example_map2 = folium.Map([39.83, -98.59], zoom_start=6)
example_map2

Next, we will create more dummy location data to simulate locations associated with different dates. Don't worry too much about the code here, but you do need to understand how the output is shaped and why it needs to be shaped like that.

In [49]:
# This cell builds together an array of initial data to display on our HeatMapwithTime. Just as before, these are dummy 
# variables that are 100 copies of the center of the USA meant to simulate different locations in the area.
# Again, we have to use lon and lat in addition to time.
np.random.seed(3141592)
initial_data = (
    np.random.normal(size=(100, 2)) * np.array([[1, 1]]) +
    np.array([[48, 5]])
)

# Create even more random lat/lon pairs and group into 100 lists
# You don't need to know how to write this code
move_data = np.random.normal(size=(100, 2)) * 0.01

data = [(initial_data + move_data * i).tolist() for i in range(100)]
data[1]

[[47.52791226223265, 4.286855296250705],
 [47.70515318785451, 4.254559938968417],
 [46.572343290775486, 3.16589036807419],
 [48.517652364172655, 3.565009343464859],
 [47.9151469875597, 4.561948133015522],
 [47.73009213894775, 6.400078703194206],
 [49.26936459309292, 3.5699714093014285],
 [47.851397175019756, 3.8319427791390397],
 [48.04477172261817, 6.316007103139816],
 [46.479416288127055, 4.682820499868778],
 [48.24059429677636, 4.343909573828612],
 [49.09604239901807, 5.094088143933742],
 [48.28316532514145, 5.662784381971345],
 [48.12325977596014, 5.782001071996817],
 [46.44347099963196, 4.770453864106374],
 [49.29117678153405, 6.136711330741487],
 [47.00836652976502, 6.286928736906839],
 [46.86855706102264, 5.113019811657515],
 [48.612612228434514, 4.677966117580723],
 [48.84324127910088, 4.533402809342304],
 [49.816563455994675, 3.759361045283218],
 [47.402859931934984, 5.299758773151624],
 [47.59727798048993, 4.887621415574294],
 [47.96991918344748, 5.560256321942765],
 [47.5909

Since we're using HeatMapWithTime, we need an extra parameter: the dates for each list of lat/lon pairs. Run the next cell to create one.

In [53]:
# Generate a set of dates for this dummy data.
# Luckily for us, when you test this out for yourself, dates came with your data set.
# You don't need to write out this code, but do look it over and see if you can understand it.
from datetime import datetime, timedelta

time_index = [
    (datetime.now() + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(len(data))
]

Finally, create the HeatMapWithTime by calling the constructor function on the data and settng the index to the set of dates you generated. Then, add it to your Map.

In [52]:
# This is the code on how to run HeatMapwithTime. Looks similar to code we saw above right?
m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)

hm = HeatMapWithTime(
    data,
    index=time_index,
    auto_play=True,
    max_opacity=0.3
)

hm.add_to(m)

m

Now try for yourself using the Berkeley `calls` data set.

The first step is to get the data into the correct format. Create a new Table with two columns: Date, containing the data in the calls "timestamp" column, and Location, containing the call location data you used to make your HeatMap (the stacked and transposed latitudes and longitudes).

Hint: check your 1-18 lab, or the Datascience Table documentation for <a href = http://data8.org/datascience/_autosummary/datascience.tables.Table.with_columns.html#datascience.tables.Table.with_columns> creating</a> and <a href = http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html> grouping</a> Tables. You're going to want to call `group` with the `list` function as the aggregator.

In [56]:
# Create a new table with the date and call location data, grouped by the date. 

locs_and_dates = Table().with_columns('Date', calls['timestamp'],
                    'Location', call_locs).group('Date', list)
locs_and_dates.show(5)

Date,Location list
2017-08-13 00:00:00,"[array([ 37.851944, -122.28349 ]), array([ 37.881142, ..."
2017-08-13 00:40:00,"[array([ 37.867373, -122.249515])]"
2017-08-13 01:00:00,"[array([ 37.876045, -122.260336])]"
2017-08-13 01:10:00,"[array([ 37.868714, -122.259189])]"
2017-08-13 04:32:00,"[array([ 37.879165, -122.302305])]"


Next, extract the dates and the grouped locations into two variables to put in your HeatMapWithTime. Note:

* HeatMapWithTime needs lists, so you'll need to convert your dates to a list using `.tolist()`
* The Table Group function converts everthing to arrays, and each array needs to be converted to a list. This is super annoying, so we've given you the code to do it. Just extract the grouped locations from the correct column and put the extracted data in the ellipses on the second line.

In [57]:
berk_dates = locs_and_dates['Date'].tolist()
berk_loc_by_date = [[x.tolist() for x in y] for y in locs_and_dates['Location list']]

Finally, create a Folium map of Berkeley, then create a <a href = https://python-visualization.github.io/folium/docs-v0.5.0/plugins.html>HeatMapWithTime</a> and add it to your Berkeley map. The call looks like `HeatMapWithTime(<grouped locations>, index=<dates>`). Click the link for more documentation. And, try adding the argument `autoplay=True`.

In [41]:
#Plot the heatmap of Berkeley crime
sf_coords = (37.76, -122.45)
sf_map2 = folium.Map(location=sf_coords, tiles='stamentoner', zoom_start=13)

berkeley_time = sorted([t[:10] for t in timestamp])

hm_sf2 = HeatMapWithTime(
    call_locs,
    index=berkeley_time,
    auto_play=True,
    max_opacity=0.3
)

hm_sf2.add_to(sf_map2)
sf_map2

In [40]:
calls.to_csv('data/berkeley_crime_0218.csv')

What conclusions can you draw from this Heatmap?

<b>Solution</b>: